# Network Processing
---

#### Run this code block by block to convert a road network(s) in ESRI Shapefile or GeoJSON format into a routable and conflated network graph to use in BikewaySim.

## Import/install the following packages:


In [1]:
import os
from pathlib import Path
import time
import geopandas as gpd

## Import Network Filter Module:

In [ ]:
from network_filter import *

## Set Directory:
### Need to modify this

In [ ]:
#make directory/pathing more intuitive later
file_dir = r"C:\Users\tpassmore6\Documents\GitHub\BikewaySimDevv" #directory of bikewaysim network processing code

#change this to where you stored this folder
os.chdir(file_dir)

## Choose study area:
#### Specify what area you want to mask the spatial data by. Only network links that are partially within the study area will be imported. Note: Network links are NOT clipped or split apart.

### Try any polygon. Just don't pick something bigger than the ARC metro region

In [ ]:
studyareafp = r'base_shapefiles/bikewaysim_study_area/bikewaysim_study_area.shp'
studyarea_name = 'bikewaysim'
#city_of_atlantafp = r'base_shapefiles/coa/Atlanta_City_Limits.shp'
#atlanta_regional_commissionfp = r'base_shapefiles/arc/arc_bounds.shp'

desired_crs = "EPSG:2240"

#add new study areas if desired
studyarea = import_study_area(studyareafp, studyarea_name, desired_crs)

In [ ]:
studyarea.explore()

## Network Mapper
- this is how network node ID's will be identified and coded
- the first letter in the network ID represents its origin network
- the second letter in the network ID represent its link type
- all numbers after that are the original network ID 

In [ ]:
network_mapper = {
    "abm": "1",
    "here": "2",
    "osm": "3",
    "original": "0",
    "generated": "1"
}

## Network Data Filepaths:

In [ ]:
abmfp = r'base_shapefiles/arc/ABM2020-TIP20-2020-150kShapefiles-outputs.gdb'
herefp = r'base_shapefiles/here/Streets.shp'
osmfp = r'base_shapefiles/osm/studyarea_links_attr.gpkg'

## Set Network Import Settings
#### Create a dictionary for use in filter networks function. 
**Should not have have to modify as is.**

In [ ]:
#defined networks
networks = ["abm","here","osm"]

#abm inputs
abm = {
       "studyarea": studyarea,
       "studyarea_name": studyarea_name,
       "networkfp": abmfp,
       "network_name": 'abm',
       "network_mapper": network_mapper,
       "A": "A",
       "B": "B",
       "layer": "DAILY_LINK"
       }

#here inputs
here = {
       "studyarea": studyarea,
       "studyarea_name": studyarea_name,
       "networkfp": herefp,
       "network_name": 'here',
       "network_mapper": network_mapper,
       "A": "REF_IN_ID",
       "B": "NREF_IN_ID",
       "layer": 0,
       "network_mapper": network_mapper
       }

osm = {
      "studyarea": studyarea,
       "studyarea_name": studyarea_name,
       "networkfp": osmfp,
       "network_name": 'osm',
       "network_mapper": network_mapper,
       "A": 'A',
       "B": 'B',
       "layer": 0,
       }

## Run Network Filter Module to Create Initial Subnetworks
From the network_filter.py file run the filter networks function. This will first import the spatial data and then filter the data into a base, road, bike, or serivce link. 
**Note: If this is the a new network that is not OSM, HERE, or ABM then specify a new filter method by going into the network_filter.py file.** Otherwise, none of the links will be filtered into road/bike/service links. **Also note: all spatial files are being projected to EPSG 2240 right now.** Need to modify later.


## Filter ABM

In [ ]:
filter_networks(**abm)

## Filter HERE

In [ ]:
filter_networks(**here)

## Filter OSM (just run this one Juwon)

In [ ]:
filter_networks(**osm)

# Everything below here still in progress

## Summurize Network Results (work in progress)

In [ ]:
from attribute_summary import * 

abm_road = import_geojson('abm', 'study_area', 'road')
osm_road = import_geojson('osm','study_area','road')
here_road = import_geojson('here', 'study_area', 'road')

#filter to only functional class, lane, and speed limit columns
abm_filt = ['NAME','FACTYPE', 'FCLASS', 'SPEED', 'SPEEDLIMIT', 'LANESTOTAL', 'LANES', 'geometry']
osm_filt = ['name', 'highway', 'maxspeed', 'lanes', 'lanes:backward', 'lanes:forward', 'geometry']
here_filt = ['ST_NAME','FUNC_CLASS', 'SPEED_CAT', 'LANE_CAT', 'PHYS_LANES' , 'TO_LANES', 'FROM_LANES', 'geometry']

abm_road = abm_road[abm_filt]
osm_road = osm_road[osm_filt]
here_road = here_road[here_filt]

## Conflation Process (work in progress)
In this step, the networks are conflated to each other by utilizing functions in the network_conflation.py module

i like to remove all of the columns that aren't related to node_id or geometry for this step. To make sure we preserve link information I also make a A_B column


there are three main function in conflation module. The first just matches nearest points.

The second will find the nearest spot on a link and create new points

The last is a function that deals with removing links/nodes that have already been considered


#only select columns that correspond to A, B, and geo

#create an A_B column

#determine what you want the base network to be, and which order you want to conflate in
#for this project, ABM was the base with HERE followed by OSM as the joining ones.




#do this step outside of the function

#Filter joining network
This should make it so that the only joining nodes that the base nodes can join to represent real intersections
<code>joining_nodes = joining_nodes[joining_nodes[f'{joining_name}_num_links'] != 2 ].reset_index(drop=True)<code>


In [2]:
from conflation_tools import *

### Links and Nodes to Conflate

In [3]:
base_name = "abm"
join_name = "here"

#road layers
base_links = gpd.read_file(r"C:/Users/tpassmore6/Documents/GitHub/BikewaySimDev/processed_shapefiles/abm/abm_bikewaysim_road_links.geojson")
base_nodes = gpd.read_file(r"C:/Users/tpassmore6/Documents/GitHub/BikewaySimDev/processed_shapefiles/abm/abm_bikewaysim_road_nodes.geojson")
join_links = gpd.read_file(r"C:/Users/tpassmore6/Documents/GitHub/BikewaySimDev/processed_shapefiles/here/here_bikewaysim_road_links.geojson")
join_nodes = gpd.read_file(r"C:/Users/tpassmore6/Documents/GitHub/BikewaySimDev/processed_shapefiles/here/here_bikewaysim_road_nodes.geojson")

#bike layers
bike_links = gpd.read_file(r'C:/Users/tpassmore6/Documents/GitHub/BikewaySimDev/processed_shapefiles/here/here_bikewaysim_bike_links.geojson')
bike_nodes = gpd.read_file(r'C:/Users/tpassmore6/Documents/GitHub/BikewaySimDev/processed_shapefiles/here/here_bikewaysim_bike_nodes.geojson')
bike_name = 'here'


### Cleaning to get rid of excess columns

In [4]:
base_links, base_nodes = cleaning_process(base_links,base_nodes,base_name)
join_links, join_nodes = cleaning_process(join_links,join_nodes,join_name)

#clean excess columns
bike_links, bike_nodes = cleaning_process(bike_links,bike_nodes,bike_name)

### Node Matching

In [5]:
#first match the nodes, can repeat this by adding in previously matched_nodes
tolerance_ft = 25
matched_nodes, unmatched_base_nodes, unmatched_join_nodes = match_nodes(base_nodes, base_name, join_nodes, join_name, tolerance_ft, prev_matched_nodes=None)

#join the matched nodes to the base nodes once done with matching
matched_nodes_final = pd.merge(base_nodes, matched_nodes, on = f'{base_name}_ID', how = "left")

1422 initial matches
There were 12 duplicates, now there are 1416 matches.
There are 103 abm nodes and 3355 here nodes remaining
1416 node pairs have been matched so far.


### Link Splitting and Add New Links and Nodes

In [6]:
#create new node and lines from the base links by splitting lines can repeat after the add_new_links_nodes function
tolerance_ft = 25
split_lines, split_nodes, unmatched_join_nodes = split_lines_create_points(unmatched_join_nodes, join_name, base_links, base_name, tolerance_ft, export = False)

#add new links and nodes to the base links and nodes created from split_lines_create_points function
new_links, new_nodes = add_new_links_nodes(base_links, matched_nodes_final, split_lines, split_nodes, base_name)

There are 1369 here points matching to 680 abm links
There are 1986 here nodes remaining
There were 2005 new lines created.


C:\Users\tpassmore6\Documents\GitHub\BikewaySimDev\conflation_tools.py:231: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  df_split = df_split.explode().reset_index(drop=True)


### Attribute Add

In [7]:
#match attribute information with greatest overlap from joining links
new_base_links_w_attr = add_attributes(new_links, base_name, join_links, join_name)

### Add rest of features

In [8]:
#add unrepresented features from joining by looking at the attributes added in prevoius step for links and the list of matched nodes
added_base_links, added_base_nodes = add_rest_of_features(new_base_links_w_attr,new_nodes,base_name,join_links,join_nodes,join_name)

C:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [9]:
added_base_links.head()

,abm_A_B,abm_line_geo,here_A_B,original_length
0,1010848_10340018,"LINESTRING (2233265.865 1384662.528, 2233329.3...",201063859187_2017289087,NaN
1,1010854_1010856,"LINESTRING (2231248.476 1375719.070, 2231166.4...",2017289109_2017289100,NaN
2,1013524_1053435,"LINESTRING (2239431.313 1375903.192, 2239373.5...",2044584013_2030636889,NaN
3,1013524_10299911,"LINESTRING (2239431.313 1375903.192, 2239452.5...",2030636889_2044584017,NaN
4,1013525_1053438,"LINESTRING (2239619.442 1375764.578, 2239568.0...",201181585260_2044584017,NaN


In [10]:
bike_nodes.geometry.geometry.name

'here_point_geo'

### Merge with other networks

In [11]:
#merge diff netwrks
tolerance_ft = 25
merged_links, merged_nodes, connections = merge_diff_networks(added_base_links, added_base_nodes, 'road', bike_links, bike_nodes, 'bike', tolerance_ft)

390 nodes already in the base network
True
here_ID_bike           False
here_num_links_bike    False
here_point_geo_bike    False
dtype: bool


UnboundLocalError: local variable 'connections' referenced before assignment

### Add reference IDs

In [ ]:
# match reference IDs based on all the id in the nodes
refid_base_links = add_reference_ids(merged_links, merged_nodes)

### Export

In [ ]:
refid_base_links.to_file(r'C:\Users\tpassmore6\Documents\GitHub\BikewaySimDev\processed_shapefiles\final_links.geojson', driver = 'GeoJSON')
merged_nodes.to_file(r'C:\Users\tpassmore6\Documents\GitHub\BikewaySimDev\processed_shapefiles\final_nodes.geojson', driver = 'GeoJSON')

## Convert for use in BikewaySim

In [ ]:
import convert_to_bikeway_sim_network

In [ ]:
#filepaths for networks
base_linksfp = r""
base_nodesfp = r""
join_linksfp = r""
join_nodesfp = r""

#column names for A, B, and/or linkID
base_mask = ['abm_A','abm_B','abm_A_B','geometry']
join_mask = ['here_A','here_B','here_A_B','geometry']

In [ ]:
#import base network
base_links = gpd.read_file(base_linksfp)
base_nodes = gpd.read_file(base_nodesfp)

#import joining network 1
join_links = gpd.read_file(join_linksfp)
join_nodes = gpd.read_file(join_nodesfp)

#import joining network 2 if applicable
#join_links2 = gpd.read_file(join_links2fp)
#join_nodes2 = gpd.read_file(join_nodes2fp)

#can add more if needed

In [ ]:
#use mask to only keep neccessary columns
base_links = base_links[base_mask]
join_links = join_links[join_mask]

#rename geometry columns
base_links = base_links.rename(columns={'geometry':'abm_geometry'})
join_links = join_links.rename(columns={'geometry':'here_geometry'})

In [ ]:
#match points


In [ ]:
from network_conflation import *

#%% Conflating HERE to OSM
# Import Links and Nodes

import matplotlib.pyplot as plt

#filepaths
abm_road, abm_road_nodes = import_links_and_nodes('abm','study_area','road')
osm_road, osm_road_nodes = import_links_and_nodes('osm','study_area','road')

here_road, here_road_nodes = import_links_and_nodes('here', 'study_area','road')


#%% filter out nodes with two connecting links

#remove nodes with only 2 connecting links
osm_filt = osm_road_nodes[osm_road_nodes[f'osm_num_links'] != 2 ]
here_filt = here_road_nodes[here_road_nodes[f'here_num_links'] != 2 ]

#intersection matching
#tolerances
tolerances = list(range(5,36,10))

#use for iterating through tolerances
matched_intersections_here = {}
matches_here = []
dup_here = []
rem_here = []
rem_2_link_here = []

matched_intersections_osm = {}
matches_osm = []
dup_osm = []
rem_osm = []
rem_2_link_osm = []

for x in tolerances:
    matched_intersections_here[x], dup_matched, matched = match_intersections(abm_road_nodes, 'abm', here_filt, 'here', x)
    matches_here.append(matched)
    dup_here.append(dup_matched)
    unmatched_abm_nodes, unmatched_here_nodes = remaining_intersections(matched_intersections_here[x], abm_road_nodes, 'abm', here_road_nodes, 'here')
    rem_here.append(len(unmatched_abm_nodes))
    rem_2_link_here.append(len(unmatched_abm_nodes[unmatched_abm_nodes['abm_num_links'] != 2]))
    
    matched_intersections_osm[x], dup_matched, matched = match_intersections(abm_road_nodes, 'abm', osm_filt, 'osm', x)
    matches_osm.append(matched)
    dup_osm.append(dup_matched)
    unmatched_abm_nodes, unmatched_osm_nodes = remaining_intersections(matched_intersections_osm[x], abm_road_nodes, 'abm', osm_road_nodes, 'osm')
    rem_osm.append(len(unmatched_abm_nodes))
    rem_2_link_osm.append(len(unmatched_abm_nodes[unmatched_abm_nodes['abm_num_links'] != 2]))

df = pd.DataFrame(list(zip(tolerances,matches_here,dup_here,rem_here,rem_2_link_here,matches_osm,dup_osm,rem_osm,rem_2_link_osm)), 
                  columns = ['tolerance', 'here match', 'here dup','here_rem','here_rem_2', 'osm match', 'osm dup', 'osm rem', 'osm rem 2'])

#plot relationship between tolerance and matches and dup_matches
plt.plot(tolerances, matches_osm) 
plt.plot(tolerances, matches_here)

#find remaining nodes and match these
unmatched_abm_nodes, unmatched_osm_nodes  = remaining_intersections(matched_intersections_here[35], abm_road_nodes, 'abm', here_road_nodes, 'here')

#%%

matched_intersections, dup_matched, matched = match_intersections(here_filt, 'here', osm_filt, 'osm', 30)
matched_intersections.to_file(r'Processed_Shapefiles/conflation_redux/matched_intersection.geojson', driver = 'GeoJSON')

#find remaining nodes and match these
unmatched_here_nodes, unmatched_osm_nodes  = remaining_intersections(matched_intersections, here_road_nodes, 'here', osm_road_nodes, 'osm')

matched_intersections_2, dup_matched, matched = match_intersections(unmatched_here_nodes, 'here', unmatched_osm_nodes, 'osm', 20)
matched_intersections_2.to_file('Processed_Shapefiles/conflation_redux/other_matched_nodes.geojson', driver = 'GeoJSON')

# add matched nodes to intersection matches
bikewaysim_nodes_v1 = matched_intersections.append(matched_intersections_2).reset_index()

# splitting link
unmatched_here_nodes, unmatched_osm_nodes  = remaining_intersections(bikewaysim_nodes_v1, here_road_nodes, 'here', osm_road_nodes, 'osm')

# find unmatched here nodes that lie on abm, find correspoding interpolated abm node, find corresponding abm link        
unmatched_osm_on, corre_here_point, corre_here_link, osm_nodes_remaining = point_on_line(unmatched_osm_nodes, "osm", here_road, "here", tolerance_ft = 20)


print(f'{len(corre_here_point)} split points found.')

here_split_multi_lines = split_by_node_to_multilinestring(corre_here_link, "here_ip_line", corre_here_point, "here_ip_point", "here")
here_split_lines, here_rest_of_lines = transform_multilinestring_to_segment(here_split_multi_lines, here_road, "here")
print(f'{len(here_split_lines)} here split lines were added.')
 
#it seems like the remaining OSM nodes either are just links that are represented in the here service network,
# dead end streets, feature difference, or a result of how the boundary was clipped





# add these split nodes and links into the conflated network
#create new ids for split nodes
new_nodes = new_ids(corre_here_point, bikewaysim_nodes_v1, 'here', 'osm')
#add split links into conflated network
bikewaysim_links_v1 = add_split_links_into_network(here_road, here_split_lines, new_nodes, 'here', 'osm')

#%%
#transfer attributes

#get ones with ref node pairs
test = (bikewaysim_links_v1['osm_A'].isna() == False) & (bikewaysim_links_v1['osm_B'].isna() == False)

#[ for x in joining_node_ids]





  

#%%

# bring in A_B id for joining network for overlapping links so that overlapping links can be brought in
# bring in any links that don't overlap with base networ
bikewaysim_links_v2 = add_in_other_links(bikewaysim_links_v1, osm_road, new_nodes, 'here', 'osm')

#future method
#transfer attribute
#test = attribute_transfer(osm_road, 'osm', new_nodes, bikewaysim_links_v1)


# add in nodes
bikewaysim_nodes_v3 = node_create_bikewaysim(bikewaysim_links_v2, osm_road_nodes, bikewaysim_nodes_v2, 'here', 'osm')

#add in bike nodes
osm_bike, osm_bike_nodes = import_links_and_nodes('osm','study_area','bike')

bikewaysim_nodes_v4 = add_bike_nodes(bikewaysim_nodes_v3, 'bikewaysim', osm_bike_nodes, 'osm', 35)

#add in bike links (this one isn't working yet)
bikewaysim_links_v3 = add_bike_links(bikewaysim_nodes_v4, bikewaysim_links_v2, 'bikewaysim', osm_bike, 'osm')
